In [2]:
# 加载库文件
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [3]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 读取mnist数据
mnist = input_data.read_data_sets("../../datasets/MNIST_data/",
                                 dtype=tf.uint8,
                                 one_hot=True)

# 获取训练数据
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1]
num_examples = mnist.train.num_examples

# 定义TFRecord地址
filename = "./output.tfrecord"
# 创建一个writer
writer = tf.python_io.TFRecordWriter(filename)
for index in range(num_examples):
    # 把每一个样例转化为一个字符串
    image_raw = images[index].tostring()
    # 将一个样例的所有的信息写入Buffer这个数据结构
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels),
        'label':_int64_feature(np.argmax(labels[index])),
        'image_raw':_bytes_feature(image_raw)
    }))
    # 将一个Example写入一个TFRecord的文件中
    writer.write(example.SerializeToString())
writer.close()
print("TFRecord文件以及保存")

Extracting ../../datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
TFRecord文件以及保存


In [ ]:
# 创建一个reader来读取TFRecord文件中的样例
reader = tf.TFRecordReader()
# 创建一个队列来维护输入文件列表
filename_queue = tf.train.string_input_producer(["Records/output.tfrecords"])
# 从文件中读出一个样例。也可以使用read_up_to函数一次性读取多个样例
_,serialized_example = reader.read(filename_queue)

# 解析读入的一个样例。如果需要解析多个样例，可以用parse_example函数
features = tf.parse_single_example(
    serialized_example,
    features={
        # Tensorflow提供两种不同的属性解析方法。
        # 一种方法是tf.FixedLenFeature,这种方法解析的结果为一个Tensor
        # 另一种方法是tf.VarLenFeature,这种方法得到的结果为SparseTensor,用于处理稀疏数据
        # 这里解析数据的格式需要和上面写入数据的格式一致
        'image_raw':tf.FixedLenFeature([],tf.string),
        'pixels':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })

# tf.decode_raw可以将字符串解析成图像对应的像素数组
images = tf.decode_raw(features['image_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)

sess = tf.Session()

# 启动多线程处理输入数据。
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

# 每次运行可以读取TFRecord文件中的一个样例
# 当所有样例都读完之后，在此样例中程序会再重头读取
for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])